In [ ]:
# 2025.4.8 tue. created by Kensuke Ohtake
# Footloose-Entrepreneur model in continuous space

using Random
using Distributions
using Plots
using Dates
using Format
using DataFrames
using CSV
using LinearAlgebra

function FE_simulation(pls)

    SD = Dates.millisecond(now())
    Random.seed!(SD) # Set seed
    
    # control parameters 
    sig = pls[1] # elasticity of substitution of manufactured goods
    tm = pls[2] # manufacturing transportation cost parameter
    
    function dist(z)
        minimum([abs(z), 2.0 * π - abs(z)])
    end
    
    N = 255 # Number of space fragments
    dx = 2.0 * π / convert(Float64, N) # size of space fragments
    dt = 0.01 # size of time step
    F = 1.0 # fixed input
    mu = 0.6 # importance of manufacturing goods
    eps = 1e-10 # error
    v = 1.0 # adjustment speed of migration
    
    x = zeros(Float64, N, 1)
    for i in 1:N
        x[i] = -π + (i-1) * dx
    end
    
    # immobile workers
    ph = ones(Float64, (N, 1)) / (2.0 * π)
    
    # Transport matrix
    Ex = zeros(Float64, (N, N))
    for i in 1:N
        for j in 1:N
            Ex[i,j] = exp(- tm * (sig - 1.0) * dist(x[i]-x[j])) # for manufacturing
        end
    end
    
    # initial values of manufacturing workers
    turb_lm = rand(Uniform(-0.0005, 0.0005), (N, 1)) # perturbation generated by random numbers
    turb_lm = turb_lm .- (sum(turb_lm) * dx) / (2.0 * π) # total equals to 0
    lm = (ones(Float64, (N, 1)) / (2.0 * π)) + turb_lm # mobile workers: homogeneous state + perturbation
    
    if sum(Int.(sign.(lm))) != N
        println("Non-negativity_is_not_satisfied_!!")
    else
        ini_lm = copy(lm)
        println("Initial total mobile workers is ", sum(ini_lm)*dx)
    end
    
    # operator for instantaneous equilibrium
    function IntegOperat(nw, immobile, mobile)# Integral operator
        # nw: nominal wage
        # immobile: phi
        # mobile: lambda
        Y = (nw .* mobile) .+ immobile
        G_integral = Ex * mobile
        output = (mu / sig) * Ex * (Y ./ G_integral)
        return output
    end
    
    # Right-hand side of the evolution equation
    function RHS(candidate_nw, immobile, mobile)
        # Candidate_nw: candidate of fixed point of instantaneous equilibrium of nominal wage
        # Inputs must be array of which shape is (I, 1) 
        # solve fixed point problem for instantaneous equilibrium
        nominalwage = copy(candidate_nw)
        while true
            old_nominalwage = copy(nominalwage)
            nominalwage = IntegOperat(old_nominalwage, immobile, mobile)
            
            if maximum(abs.(nominalwage - old_nominalwage)) < eps
                break
            end
        end

        # price index
        G = ((1.0 / F) * (Ex * mobile * dx)) .^ (1.0 / (1.0 - sig))

        # real wage
        rw = nominalwage .* (G .^ (-mu)) # real wage
    
        # Right-hand side of the evolution equation
        rhs = v * ((rw .- dot(rw, mobile) * dx) .* mobile)
    
        return rhs, nominalwage
    end
    
    #= time evolution =#

    # time step
    t = 0

    # initialize candidate for nominal wage of fixed point
    w = ones(Float64, (N, 1)) 

    # Initialize update amount of lambda
    dlm = zeros(Float64, (N, 1))

    # Iterate until a steady-state solution is reached
    while true

        # Save previous values
        lm_old = copy(lm)
        w_old = copy(w)
        
        # 1-st order Runge Kutta (=Euler) method for the replicator equation 
        k_1, w = RHS(w_old, ph, lm)
        dlm = dt * k_1
        lm = lm_old + dlm
        
        # check nonnegativity
        if sum(Int.(sign.(lm))) != N
            println("Non-negativity_is_not_satisfied_!!")
            break
        end
        
        # check conservation of population
        if 1.0 - 1e-10 > sum(lm) * dx
            println("Conservation_error_!!")
            break
        elseif sum(lm) * dx > 1.0 + 1e-10
            println("Conservation_error_!!")
            break
        end

        # stationary solution condition
        if maximum(abs.(dlm)) < eps
            break
        end
    
        t += 1
    
    end

    #= Obtain stationary solutions =#
    stationary_w = copy(w)
    while true
        old_stationary_w = copy(stationary_w)
        stationary_w = IntegOperat(old_stationary_w, ph, lm)
        if maximum(abs.(stationary_w - old_stationary_w)) < eps
            break
        end
    end

    nominal_wage = stationary_w
    price_index = ((1 / F) * (Ex * lm) * dx) .^ (1.0 / (1.0 - sig))
    real_wage = nominal_wage .* (price_index .^ (-mu))

    println("Final total mobile workers is ", sum(lm) * dx)

    # Plot
    DateTime = Dates.format(now(), "yyyymmddHHMMSS")# get datetime
    plt_x = append!(reshape(x, N), π)
    plt_ini_lm = append!(reshape(ini_lm, N), ini_lm[1])
    plt_lm = append!(reshape(lm, N), lm[1])
    plt_nominal_wage = append!(reshape(nominal_wage, N), nominal_wage[1])
    plt_price_index = append!(reshape(price_index, N), price_index[1])
    plt_real_wage = append!(reshape(real_wage, N), real_wage[1])
    
    filename_lm = format("{}-pop_tau{}sigma{}mu{}N{}t{}sd{}", DateTime, tm, sig, mu, N, t, SD)
    newfilename_lm = replace(filename_lm, "."=>"p")
    plot(plt_x, plt_lm, label="final", dpi=300)
    plot!(plt_x, plt_ini_lm, title="mobile workers", label="initial", dpi=300)
    savefig(newfilename_lm * ".png")
    
    filename_rw = format("{}-rw_tau{}sigma{}mu{}N{}t{}sd{}", DateTime, tm, sig, mu, N, t, SD)
    newfilename_rw = replace(filename_rw, "."=>"p")
    plot(plt_x, plt_real_wage, title="real wage", label="", dpi=300)
    savefig(newfilename_rw * ".png")
    
    # Output and save data as csv
    df = DataFrame(coordinates = plt_x,
        initial_pop_lm = plt_ini_lm,
        final_pop_lam = plt_lm,
        nominal_wage = plt_nominal_wage,
        price_index = plt_price_index,
        real_wage = plt_real_wage) # Save data as dataframe
    filename = format("{}_tau{}sigma{}mu{}N{}sd{}", DateTime, tm, sig, mu, N, SD)
    newfilename = replace(filename, "."=>"p")
    CSV.write(newfilename * ".csv", df)# Write into csv
end


In [ ]:
# Loop
for sig in [2.7, 2.5, 2.4, 2.2, 2.0, 1.7]
    params = [sig, 2.0] # set parameters [sigma, tau]
    for itr in 1:5
        FE_simulation(params)
    end
end
